**ES e Redes Neurais**

**Autor**: Iran Freitas Ribeiro

**Disciplina**: Computação Natural

**Professor**: Renato A. Krohling

Implementação da rede neural baseada em [How to Code a Neural Network with Backpropagation In Python (from scratch)
](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/)

https://colab.research.google.com/drive/1ku3LvrqovKOzeCTkW6bLHYxNKMaG4KFv

In [1]:
import numpy as np
from math import exp
from random import seed
from random import random
from numpy import argsort
from numpy.random import uniform

# Feed Foward NN

In [2]:
# TODO: inicializar pesos com uma distribuição uniforme
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs+1)]} for j in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden+1)]} for j in range(n_outputs)]
    network.append(output_layer)
    return network

def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

def transfer(activation):
    return 1.0/(1.0+exp(-activation))

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def transfer_derivative(output):
    return output * (1.0 - output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for  neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs))    :
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print ("Epoch={}, lrate={:.3f}, error={:.3f}".format(epoch, l_rate, sum_error))

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# ES

 - 

In [113]:
####
# ES
####
def initialize_population(n_inputs, n_hidden, n_outputs, size):
    """
    Inicia a população
    """
    population = list()
    for i in range(size):
        network = list()
        hidden_layer = [[np.random.randn() for i in range(n_inputs+1)] for j in range(n_hidden)]
        network.append(hidden_layer)
        output_layer = [[np.random.randn()  for i in range(n_hidden+1)] for j in range(n_outputs)]
        network.append(output_layer)
        population.append(network)
    
    return population

def eval_individual(p, train, n_outputs):
    sum_error = 0
    for row in train:
        network = get_weigths(p)
        outputs = forward_propagate(network, row)            
        expected = [0 for i in range(n_outputs)]
        #print (expected)        
        expected[int(row[-1])-1] = 1
        sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
    return sum_error


def get_weigths(individual):
    network = list()
    hidden_layer = [{'weights':j} for j in individual[0]]
    network.append(hidden_layer)
    output_layer = [{'weights':j} for j in individual[1]]
    network.append(output_layer)
    return network

def marriage(pais,p):
    """
    Escolhe os pais para o casamento
    
    Params:
        - pais: população de pais
        - p: numero de pais escolhidos para o casamento
    return:
        - pais_escolhidos: lista de pais escolhidos
        - indices_pais: indices dos pais escolhidos
    """
    # indice dos pais
    pais_ids = np.arange(0,len(pais))
    # escolha p idices aleatorioamente
    indices_pais = np.random.choice(pais_ids,size=p, replace=False)
    # seleciona os p pais referentes aos indices
    pais_escolhidos = [pais[i] for i in indices_pais]

    return pais_escolhidos

def recombination(pais):
    # TODO: REFAZER RECOMBINAÇÃO (SEPARAR PESOS DE BIAS E ALTERAR DE UMA VEZ)
    """
    Gera um filho a partir da recombinação de p pais

    Params:
        - pais: pais para serem recombinados

    Return:
        - filho: filho gerado a partir da recombinação
    """
    filho = list()
    # cria um filho com 0 valores iguais à um pai
    h_weights = np.zeros_like(pais[0][0])
    o_weights = np.zeros_like(pais[0][1])
    p = len(pais)

    # recombinação camadas escondidas
    for h in range(len(h_weights)):
        # escolhe o pai
        j = np.random.randint(0,high=p)     
        # escolhe a informação do pai que sera transmitida
        k = np.random.randint(0,h_weights[h].shape[0]-1)
        pai = pais[j]
        h_weights[:,h] = np.array(pai[0])[:, :2][:,k]
        h_weights[h][-1] = pai[0][h][-1]
        
    # recombinação camadas de saida
    # percorre cada saída
    for o in range (len(o_weights)):        
        # escolhe o pai
        j = np.random.randint(0,high=p)          
        # escolhe a informação do pai que sera transmitida
        k = np.random.randint(0,o_weights.shape[1]-1)
        # transmite a informação para o filho
        pai = pais[j]
        o_weights[:,o] = np.array(pai[1])[:, :2][:,k]
        o_weights[o][-1] = pai[1][o][-1]
            
    filho.append(h_weights)
    filho.append(o_weights)
    return filho

def mutation(filho, a):
    """
    Aplica a recombinação no filho

    Params:
        - filho: individuo a ser recombinado
        - a: força aplicada na mutação (step size)
    
    return:
        - filho_: filho mutado

    """
    f_h = filho[0]+np.random.normal(0, a, size=np.array(filho[0]).shape[1])
    f_o = filho[1]+np.random.normal(0, a, size=np.array(filho[1]).shape[1])
    filho_ = [f_h, f_o]
    return filho_

def train_network_ES(train=None, n_inputs=None, n_hidden=1, n_epoch=50, n_outputs=1, mu=60, lam=100, p=3, a=0.015,lograte=10):
    """
    Objetivo: atualizar os pesos, usando ES, de forma que sum_error seja minimizado
    Para isso:
        - Precisa de uma população de individuos
        - Cada individuo armazena todos os pesos da rede
    """
    population = initialize_population(n_inputs, n_hidden, n_outputs, mu)
    best, best_score = None, np.inf
    for epoch in range(n_epoch):
        children = list()
        scores_children = list()
        for l in range(lam):
            pais_marriage = marriage(population, p)
            filho = recombination(pais_marriage)
            filho = mutation(filho, a)
            score_filho = eval_individual(filho, train, n_outputs)

            if score_filho < best_score:
                best_score = score_filho
                best = filho
            # insere score do filho na lista de scores
            scores_children.append(score_filho)
            # insere filho na lista de filhos
            children.append(filho)

        # ordena os scores do melhor para o maior
        ranks = argsort(scores_children)
        # seleciona os mu melhores filhos
        selected_children = [children[ranks[k]] for k,_ in enumerate(ranks[:mu])]
        population = selected_children
        if epoch%lograte==0:
            print ("Epoch={} bestscore={}".format(epoch, best_score))
    return best

# Testes

In [25]:
seed(42)
nn = initialize_network(2,2,2)
for layer in nn:
    print (layer)

[{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926]}, {'weights': [0.22321073814882275, 0.7364712141640124, 0.6766994874229113]}]
[{'weights': [0.8921795677048454, 0.08693883262941615, 0.4219218196852704]}, {'weights': [0.029797219438070344, 0.21863797480360336, 0.5053552881033624]}]


In [29]:
nn

[[{'weights': [0.6394267984578837,
    0.025010755222666936,
    0.27502931836911926]}],
 [{'weights': [0.22321073814882275, 0.7364712141640124]},
  {'weights': [0.6766994874229113, 0.8921795677048454]}]]

In [31]:
seed(42)
i = initialize_population(2,1,2,0)
i

[[[0.6394267984578837, 0.025010755222666936, 0.27502931836911926]],
 [[0.22321073814882275, 0.7364712141640124],
  [0.6766994874229113, 0.8921795677048454]]]

In [32]:
get_weigths(i)

[{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926]},
 [{'weights': [0.22321073814882275, 0.7364712141640124]},
  {'weights': [0.6766994874229113, 0.8921795677048454]}]]

In [12]:
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [15]:
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print (layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


In [5]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
# n_inputs = len(dataset[0]) - 1
# n_outputs = len(set([row[-1] for row in dataset]))
# network = initialize_network(n_inputs, 2, n_outputs)
# train_network(network, dataset, 0.5, 20, n_outputs)
# for layer in network:
# 	print(layer)

# Testes ES

In [258]:
import pandas as pd

In [260]:
df_wheat = pd.read_csv("wheat-seeds.csv", header=None)
df_wheat.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [261]:
data = df_wheat.values

In [262]:
data

array([[15.26  , 14.84  ,  0.871 , ...,  2.221 ,  5.22  ,  1.    ],
       [14.88  , 14.57  ,  0.8811, ...,  1.018 ,  4.956 ,  1.    ],
       [14.29  , 14.09  ,  0.905 , ...,  2.699 ,  4.825 ,  1.    ],
       ...,
       [13.2   , 13.66  ,  0.8883, ...,  8.315 ,  5.056 ,  3.    ],
       [11.84  , 13.21  ,  0.8521, ...,  3.598 ,  5.044 ,  3.    ],
       [12.3   , 13.34  ,  0.8684, ...,  5.637 ,  5.063 ,  3.    ]])

In [116]:
seed(42)
best_nn = train_network_ES(train=dataset, n_inputs=2, n_hidden=1, n_epoch=1000, n_outputs=2, mu=20, lam=60, p=2, a=0.015, lograte=20)

Epoch=0 bestscore=4.604610226836142
Epoch=20 bestscore=4.450900962735713
Epoch=40 bestscore=4.364658810516769
Epoch=60 bestscore=4.317696569821685
Epoch=80 bestscore=4.28591458676236
Epoch=100 bestscore=4.245765691833026
Epoch=120 bestscore=4.218714312938251
Epoch=140 bestscore=4.149214378235007
Epoch=160 bestscore=4.117818192837373
Epoch=180 bestscore=4.072454151843454
Epoch=200 bestscore=4.033155817077433
Epoch=220 bestscore=4.0008205411283635
Epoch=240 bestscore=3.987770390635367
Epoch=260 bestscore=3.9564974413492893
Epoch=280 bestscore=3.9265187577622234
Epoch=300 bestscore=3.8856006415500626
Epoch=320 bestscore=3.8599050837457294
Epoch=340 bestscore=3.84887073313963
Epoch=360 bestscore=3.8411993158061764
Epoch=380 bestscore=3.812015939362796
Epoch=400 bestscore=3.8029175470020045
Epoch=420 bestscore=3.7828042804004007
Epoch=440 bestscore=3.761101959791826
Epoch=460 bestscore=3.758628846021407
Epoch=480 bestscore=3.750654954576266
Epoch=500 bestscore=3.7411796091056844
Epoch=520 b

In [112]:
best_nn

[array([[-4.36011778,  0.22614585, 17.78313421]]),
 array([[-0.28764602,  0.08232246],
        [ 0.40800815, -0.27157236]])]

In [274]:
datascaled.shape

(210, 8)

In [53]:
population = initialize_population(2, 2, 2, 2)
pai = population[0]


In [54]:
np.array(pai[0])[:, :2][:, 0]

array([0.98744867, 0.02768785])

In [55]:
h_weights = np.zeros_like(pai[0])

In [75]:
pai[0]

[[0.9874486700702166, 0.3991943958495391, 0.8604288666460781],
 [0.027687849234446116, 0.46497268948191406, 0.15957960139626004]]

In [83]:
for h in range(len(h_weights)):
    # escolhe o pai
    j = np.random.randint(0,high=2)     
    # escolhe a informação do pai que sera transmitida
    k = np.random.randint(0,h_weights[h].shape[0]-1)
    
    print (k)
    h_weights[:,h] = np.array(np.array(pai[0]))[:, :2][:,k]
    h_weights[h][-1] = pai[0][h][-1]

1
0


In [84]:
h_weights

array([[0.3991944 , 0.98744867, 0.86042887],
       [0.46497269, 0.02768785, 0.1595796 ]])

In [58]:
np.array(pai[0])[:, :2][:, 0]

array([0.98744867, 0.02768785])

In [61]:
h_weights[:, 0] = np.array(pai[0])[:, :2][:, 0]

In [62]:
h_weights

array([[0.98744867, 0.        , 0.        ],
       [0.02768785, 0.        , 0.        ]])

In [18]:
h_weights = np.zeros_like(population[0][0])

In [20]:
h_weights[0].shape

(3,)

In [289]:
o_weights = np.zeros_like(population[0][1])

In [17]:
np.zeros_like(population[0][0]).shape

(2, 3)

In [291]:
recombination()

(3, 2)

In [303]:
seed(42)
best_nn = train_network_ES(train=datascaled, n_inputs=7, n_hidden=1, n_epoch=150, n_outputs=3, mu=20, lam=60, p=15, a=0.3, lograte=20)

Epoch=0 bestscore=148.08980043394718
Epoch=20 bestscore=134.0781718700628
Epoch=40 bestscore=133.27824922363064
Epoch=60 bestscore=133.27824922363064
Epoch=80 bestscore=133.27824922363064
Epoch=100 bestscore=133.27824922363064
Epoch=120 bestscore=133.27824922363064
Epoch=140 bestscore=133.27824922363064


In [142]:
from sklearn.preprocessing import MinMaxScaler

In [226]:
nn = get_weigths(best_nn)
predict(nn, dataset[0])

1

In [225]:
dataset[1]

array([1.46548937, 2.36212508, 0.        ])

In [146]:
dataset

array([[ 2.7810836 ,  2.550537  ,  0.        ],
       [ 1.46548937,  2.36212508,  0.        ],
       [ 3.39656169,  4.40029353,  0.        ],
       [ 1.38807019,  1.85022032,  0.        ],
       [ 3.06407232,  3.00530597,  0.        ],
       [ 7.62753121,  2.75926224,  1.        ],
       [ 5.33244125,  2.08862677,  1.        ],
       [ 6.92259672,  1.77106367,  1.        ],
       [ 8.67541865, -0.24206865,  1.        ],
       [ 7.67375647,  3.50856301,  1.        ]])

In [268]:
X_train = MinMaxScaler().fit_transform(data[:, [0,1,2,3,4,5,6]])

In [263]:
data.shape

(210, 8)

In [269]:
datascaled = np.concatenate([X_train,data[:,[-1]]], axis=1)